In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"korphai","key":"9b2f5c8b339ac13b626c41a010328aee"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!pip install -q kaggle


In [5]:
!kaggle datasets download -d ddosad/ecommerce-customer-service-satisfaction

Dataset URL: https://www.kaggle.com/datasets/ddosad/ecommerce-customer-service-satisfaction
License(s): other


In [9]:
!unzip ecommerce-customer-service-satisfaction.zip

Archive:  ecommerce-customer-service-satisfaction.zip
  inflating: Customer_support_data.csv  


In [6]:
#/content/Customer_support_data.csv

In [7]:
!pip install findspark

In [10]:
# import all
from pyspark.sql.types import *
from pyspark.sql.functions import *

# import select functions and types
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import floor, round

# Import SparkSession
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("CustomerSupportAnalysis").getOrCreate()


# Assign this variable your full volume file path
volume_file_path = "/content/Customer_support_data.csv"

df_csv = (spark.read
  .format("csv")
  .option("header", True)
  .option("inferSchema", True)
  .load(volume_file_path)
)
display(df_csv)

DataFrame[Unique id: string, channel_name: string, category: string, Sub-category: string, Customer Remarks: string, Order_id: string, order_date_time: string, Issue_reported at: string, issue_responded: string, Survey_response_Date: string, Customer_City: string, Product_category: string, Item_price: string, connected_handling_time: string, Agent_name: string, Supervisor: string, Manager: string, Tenure Bucket: string, Agent Shift: string, CSAT Score: string]

In [ ]:
df_csv.limit(5).show()

In [13]:
df_csv.describe()

DataFrame[summary: string, Unique id: string, channel_name: string, category: string, Sub-category: string, Customer Remarks: string, Order_id: string, order_date_time: string, Issue_reported at: string, issue_responded: string, Survey_response_Date: string, Customer_City: string, Product_category: string, Item_price: string, connected_handling_time: string, Agent_name: string, Supervisor: string, Manager: string, Tenure Bucket: string, Agent Shift: string, CSAT Score: string]

In [14]:
df_csv.columns

['Unique id',
 'channel_name',
 'category',
 'Sub-category',
 'Customer Remarks',
 'Order_id',
 'order_date_time',
 'Issue_reported at',
 'issue_responded',
 'Survey_response_Date',
 'Customer_City',
 'Product_category',
 'Item_price',
 'connected_handling_time',
 'Agent_name',
 'Supervisor',
 'Manager',
 'Tenure Bucket',
 'Agent Shift',
 'CSAT Score']

In [45]:
from pyspark.sql.functions import unix_timestamp
df_csv = df_csv.withColumn("Issue_reported at", to_timestamp(col("Issue_reported at"), "dd/MM/yyyy HH:mm"))
df_csv = df_csv.withColumn("issue_responded", to_timestamp(col("issue_responded"), "dd/MM/yyyy HH:mm"))
df_csv.select("Issue_reported at", "issue_responded").show()
df_csv = df_csv.withColumn("response_time_minutes",
                   (col("issue_responded").cast("long") - col("Issue_reported at").cast("long")) / 60)
df_csv.select("Issue_reported at", "issue_responded", "response_time_minutes").show()


+-------------------+-------------------+
|  Issue_reported at|    issue_responded|
+-------------------+-------------------+
|2023-08-01 11:13:00|2023-08-01 11:47:00|
|2023-08-01 12:52:00|2023-08-01 12:54:00|
|2023-08-01 20:16:00|2023-08-01 20:38:00|
|2023-08-01 20:56:00|2023-08-01 21:16:00|
|2023-08-01 10:30:00|2023-08-01 10:32:00|
|2023-08-01 15:13:00|2023-08-01 18:39:00|
|2023-08-01 15:31:00|2023-08-01 23:52:00|
|2023-08-01 16:17:00|2023-08-01 16:23:00|
|2023-08-01 21:03:00|2023-08-01 21:07:00|
|2023-08-01 23:31:00|2023-08-01 23:36:00|
|2023-08-02 18:14:00|2023-08-02 18:16:00|
|2023-08-02 10:44:00|2023-08-02 11:14:00|
|2023-08-01 10:09:00|2023-08-01 10:12:00|
|2023-08-01 10:15:00|2023-08-01 11:21:00|
|2023-08-02 11:26:00|2023-08-02 11:44:00|
|2023-08-02 19:56:00|2023-08-02 20:06:00|
|2023-08-01 09:01:00|2023-08-01 09:03:00|
|2023-08-01 10:00:00|2023-08-01 10:04:00|
|2023-08-01 21:05:00|2023-08-01 21:07:00|
|2023-08-02 20:03:00|2023-08-02 20:05:00|
+-------------------+-------------

In [50]:
df_csv.groupBy("category").agg({"response_time_minutes": "avg"}).show()

+------------------+--------------------------+
|          category|avg(response_time_minutes)|
+------------------+--------------------------+
|     Order Related|        112.75397406625598|
|  Payments related|          89.2506448839209|
|           Returns|          150.337014945683|
|      Cancellation|        158.92992766726945|
|       App/website|        254.07142857142858|
| Shopzilla Related|        101.72922636103152|
| Offers & Cashback|        179.58246346555325|
|    Refund Related|        143.29384615384615|
|   Product Queries|        154.66784069357897|
|            Others|        237.95959595959596|
|Onboarding related|         40.55384615384615|
|          Feedback|        142.69877942458587|
+------------------+--------------------------+



In [51]:
df_csv.limit(5).show()

+--------------------+------------+---------------+--------------------+----------------+--------------------+---------------+-------------------+-------------------+--------------------+-------------+----------------+----------+-----------------------+-------------------+--------------+---------------+---------------+-----------+----------+-------------+---------------------+
|           Unique id|channel_name|       category|        Sub-category|Customer Remarks|            Order_id|order_date_time|  Issue_reported at|    issue_responded|Survey_response_Date|Customer_City|Product_category|Item_price|connected_handling_time|         Agent_name|    Supervisor|        Manager|  Tenure Bucket|Agent Shift|CSAT Score|response_time|response_time_minutes|
+--------------------+------------+---------------+--------------------+----------------+--------------------+---------------+-------------------+-------------------+--------------------+-------------+----------------+----------+-----------

In [54]:
df_buckets = df_csv.withColumn("response_bucket",
    when(col("response_time_minutes") < 5, "<5 min")
    .when(col("response_time_minutes") < 30, "5–30 min")
    .when(col("response_time_minutes") < 60, "30–60 min")
    .otherwise(">60 min")
)

# 2. Pivot to count incidents per time bucket per agent
pivot_df = df_buckets.groupBy("Agent_name").pivot("response_bucket").count()

# 3. Show the pivot table
pivot_df.show()


+-----------------+---------+--------+------+-------+
|       Agent_name|30–60 min|5–30 min|<5 min|>60 min|
+-----------------+---------+--------+------+-------+
|   Desiree Torres|        4|      19|    23|     14|
|   Carla Anderson|        3|       7|    19|      7|
|    Virginia Lane|        5|      28|    63|     15|
|  Christine Myers|        6|      20|    28|     17|
|        John Dean|     NULL|       9|    10|      2|
|     Amanda Sloan|        4|      27|    27|     19|
| Alexandra Harris|        3|       2|    14|      8|
|       John Wells|        6|      23|    28|     23|
|      Jenna Rivas|        8|       9|    31|      9|
| Andrea Rasmussen|        3|       9|     9|      7|
|Gregory Davenport|        6|      22|    30|     14|
|    Eric Williams|     NULL|      10|     7|      5|
|     Alyssa Jones|     NULL|       8|    13|      4|
|      Gregg Perez|        4|      13|    30|     13|
|    Donna Jenkins|        5|      21|    22|     16|
| Andrea Wilkerson|        3

In [55]:
pivot_df.write.csv("pivot_table.csv")

In [56]:
from pyspark.sql.functions import udf

In [ ]:
from pyspark.sql.types import StringType

def response_time()
  for i in

size_category = udf(lambda x: "Large" if x > 4500 else "Small", StringType())
df.withColumn("size_class", size_category(df.body_mass_g)).show()

In [59]:
from pyspark.sql.functions import greatest

pivot_df.withColumn("the most range of response time", greatest(col("30–60 min"), col("5–30 min"), col("<5 min"),col(">60 min"))).show()

+-----------------+---------+--------+------+-------+-------------------------------+
|       Agent_name|30–60 min|5–30 min|<5 min|>60 min|the most range of response time|
+-----------------+---------+--------+------+-------+-------------------------------+
|   Desiree Torres|        4|      19|    23|     14|                             23|
|   Carla Anderson|        3|       7|    19|      7|                             19|
|    Virginia Lane|        5|      28|    63|     15|                             63|
|  Christine Myers|        6|      20|    28|     17|                             28|
|        John Dean|     NULL|       9|    10|      2|                             10|
|     Amanda Sloan|        4|      27|    27|     19|                             27|
| Alexandra Harris|        3|       2|    14|      8|                             14|
|       John Wells|        6|      23|    28|     23|                             28|
|      Jenna Rivas|        8|       9|    31|      9| 

In [62]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# Initialize SparkSession
spark = SparkSession.builder.appName("WindowExample").getOrCreate()

# Example data
data = [
    ("Alice", "2025-05-01", 100),
    ("Bob", "2025-05-01", 150),
    ("Alice", "2025-05-02", 200),
    ("Bob", "2025-05-02", 50),
    ("Alice", "2025-05-03", 300),
    ("Bob", "2025-05-03", 100)
]

# Create DataFrame
columns = ["salesperson", "date", "sales"]
df = spark.createDataFrame(data, columns)

# Define window specification
window_spec = Window.partitionBy("salesperson").orderBy("date")

# Calculate cumulative sum of sales
df_with_cumsum = df.withColumn("cumulative_sales", F.sum("sales").over(window_spec))

# Show result
df_with_cumsum.show()


+-----------+----------+-----+----------------+
|salesperson|      date|sales|cumulative_sales|
+-----------+----------+-----+----------------+
|      Alice|2025-05-01|  100|             100|
|      Alice|2025-05-02|  200|             300|
|      Alice|2025-05-03|  300|             600|
|        Bob|2025-05-01|  150|             150|
|        Bob|2025-05-02|   50|             200|
|        Bob|2025-05-03|  100|             300|
+-----------+----------+-----+----------------+

